In [1]:
include("../src/CPDs/cpds.jl")
# using .CPDs

sub2ind_vec (generic function with 1 method)

In [2]:
df = DataFrame(a=randn(100))
cpd = fit(StaticCPD{Normal}, df, :a)

StaticCPD{Distributions.Normal{Float64}}(CPDCore{Distributions.Normal{Float64}}(:a,Symbol[],Distributions.Normal{Float64}(μ=0.004495360279022373, σ=1.054700819139596)))

In [3]:
name(cpd)

:a

In [4]:
parents(cpd)

0-element Array{Symbol,1}

In [5]:
distribution(cpd)

Distributions.Normal{Float64}(μ=0.004495360279022373, σ=1.054700819139596)

In [6]:
condition!(cpd, Dict{Symbol, Any}())

Distributions.Normal{Float64}(μ=0.004495360279022373, σ=1.054700819139596)

In [7]:
pdf(cpd, Dict{Symbol, Any}(:a=>0.5))

0.33872915098328266

In [9]:
pdf(cpd, Dict{Symbol, Any}(:a=>0.0))

0.378248171891748